In [13]:
import json, random
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks


In [14]:
# Cell 1 ── Imports & Paths
import json, pandas as pd, numpy as np, tensorflow as tf
from pathlib import Path
from sklearn.model_selection import train_test_split

# Where your allergy images live:
IMG_DIRS    = [
    Path("/mnt/ssd1/saumia/data/images/imgs_part_1"),
    Path("/mnt/ssd1/saumia/data/images/imgs_part_2"),
    Path("/mnt/ssd1/saumia/data/images/imgs_part_3"),
]


In [22]:
# Cell 2 ── Build & Clean DataFrame (final, with NaN‐drop before split)

import json
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

# 1) Read your metadata CSV
meta_df = pd.read_csv("/mnt/ssd1/saumia/data/text/metadata.csv")

# 2) Gather rows by matching image filenames to metadata
rows = []
for img_dir in IMG_DIRS:
    for p in img_dir.glob("*.png"):
        parts = p.stem.split("_")
        if len(parts) < 4:
            continue
        try:
            lesion_id = int(parts[2])
        except ValueError:
            continue
        m = meta_df[meta_df["lesion_id"] == lesion_id]
        if m.empty:
            continue
        m = m.iloc[0]
        row = {
            "path": str(p),
            "label": m["diagnostic"],
            "age": m["age"],
            "diameter_1": m["diameter_1"],
            "diameter_2": m["diameter_2"],
            "gender_M": 1.0 if str(m["gender"]).upper() == "MALE" else 0.0,
            "region": m["region"],
        }
        # six boolean cols
        for col in ["itch", "bleed", "elevation", "changed", "hurt", "grew"]:
            row[col] = 1.0 if bool(m.get(col)) else 0.0
        rows.append(row)

df = pd.DataFrame(rows)
assert not df.empty, "No images matched metadata!"

# 3) Drop nuisance features
constant_cols = ["itch", "bleed", "elevation", "changed", "hurt", "grew"]
df = df.drop(columns=constant_cols, errors="ignore")

# 4) Normalize continuous features to [0,1]
for col in ["age", "diameter_1", "diameter_2"]:
    if col in df:
        mn, mx = df[col].min(), df[col].max()
        df[col] = (df[col] - mn) / (mx - mn + 1e-8)

# 5) One‐hot encode the region column
df = pd.get_dummies(df, columns=["region"], dtype="float32")

# 6) Create label map and feature‐column list
classes   = sorted(df["label"].unique())
label_map = {c: i for i, c in enumerate(classes)}
feat_cols = [c for c in df.columns if c not in {"path", "label"}]

df["label_id"] = df["label"].map(label_map)

# 7) Persist your mappings
with open("label_map_image.json", "w")    as f: json.dump(label_map, f, indent=2)
with open("feature_cols_image.json", "w") as f: json.dump(feat_cols, f, indent=2)

# 8) Drop any rows with missing feature or label values
df = df.dropna(subset=feat_cols + ["label_id"]).reset_index(drop=True)

# 9) Stratified train/validation split
train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["label"],
    random_state=42
)

# 10) Ensure correct dtypes
train_df[feat_cols] = train_df[feat_cols].astype("float32")
val_df[feat_cols]   = val_df[feat_cols].astype("float32")
train_df["label_id"] = train_df["label_id"].astype("int32")
val_df["label_id"]   = val_df["label_id"].astype("int32")

print(f"{len(classes)} classes  |  {len(train_df)} train  /  {len(val_df)} val samples")


6 classes  |  1184 train  /  297 val samples


In [23]:
train_df.describe()

,age,diameter_1,diameter_2,gender_M,region_ABDOMEN,region_ARM,region_BACK,region_CHEST,region_EAR,region_FACE,region_FOOT,region_FOREARM,region_HAND,region_LIP,region_NECK,region_NOSE,region_SCALP,region_THIGH,label_id
count,1184.000000,1184.000000,1184.000000,1184.000000,1184.000000,1184.000000,1184.00000,1184.000000,1184.000000,1184.000000,1184.000000,1184.000000,1184.000000,1184.000000,1184.000000,1184.000000,1184.000000,1184.000000,1184.000000
mean,0.650031,0.117426,0.124278,0.493243,0.009291,0.086993,0.11402,0.128378,0.033784,0.231419,0.010135,0.130068,0.048142,0.014358,0.059966,0.096284,0.003378,0.033784,1.461993
std,0.166620,0.086309,0.082412,0.500166,0.095979,0.281944,0.31797,0.334652,0.180749,0.421918,0.100204,0.336520,0.214156,0.119012,0.237525,0.295105,0.058050,0.180749,1.382260
min,0.079545,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.545455,0.070000,0.071429,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.659091,0.100000,0.100000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.772727,0.150000,0.142857,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000


In [24]:
# Cell 3 ── tf.data Pipelines
IMG_SIZE   = (224,224)
BATCH_SIZE = 16
AUTOTUNE   = tf.data.AUTOTUNE

def preprocess_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    return tf.keras.applications.efficientnet.preprocess_input(img)

def make_dataset(df, shuffle=True):
    paths  = df["path"].values
    metas  = df[feat_cols].values
    labels = df["label_id"].values
    ds = tf.data.Dataset.from_tensor_slices((paths, metas, labels))
    if shuffle:
        ds = ds.shuffle(len(df), seed=42)
    def _load(path, meta, label):
        return {"image": preprocess_image(path), "meta": meta}, label
    ds = ds.map(_load, num_parallel_calls=AUTOTUNE)
    return ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

train_ds = make_dataset(train_df, shuffle=True)
val_ds   = make_dataset(val_df,   shuffle=False)
# Clean and coerce feature types
for col in feat_cols:
    train_df[col] = pd.to_numeric(train_df[col], errors="coerce")
    val_df[col]   = pd.to_numeric(val_df[col], errors="coerce")

# Drop rows with missing features
train_df = train_df.dropna(subset=feat_cols + ["label_id"]).reset_index(drop=True)
val_df   = val_df.dropna(subset=feat_cols + ["label_id"]).reset_index(drop=True)
print(f"✅ Dropped rows with missing features: now {len(train_df)} train / {len(val_df)} val")

# Ensure all features are float32
train_df[feat_cols] = train_df[feat_cols].astype("float32")
val_df[feat_cols]   = val_df[feat_cols].astype("float32")

# Optional: Clip to [0, 1] for safety
train_df[feat_cols] = train_df[feat_cols].clip(0.0, 1.0)
val_df[feat_cols]   = val_df[feat_cols].clip(0.0, 1.0)


# sanity check
for (batch_x, batch_y) in train_ds.take(1):
    print("Image batch shape:", batch_x["image"].shape)
    print("Meta batch shape:",  batch_x["meta"].shape)
    print("Labels:", batch_y.numpy()[:8])


✅ Dropped rows with missing features: now 1184 train / 297 val
Image batch shape: (16, 224, 224, 3)
Meta batch shape: (16, 18)
Labels: [3 2 1 3 1 1 1 0]


2025-05-23 05:52:43.510408: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [25]:
# Cell 4 ── Build & Compile the Model (Improved Version)
from tensorflow.keras import layers, Model
import tensorflow as tf

# Load EfficientNetB0 with pretrained ImageNet weights
base_img = tf.keras.applications.EfficientNetB0(
    include_top=False, pooling="avg", weights="imagenet")
base_img.trainable = False  # Freeze to prevent overfitting on small data

# Inputs
img_in  = layers.Input(shape=IMG_SIZE + (3,), name="image")
meta_in = layers.Input(shape=(len(feat_cols),), name="meta")

# Image path
x1 = base_img(img_in)

# Metadata path
x2 = layers.BatchNormalization()(meta_in)
x2 = layers.Dense(64, activation="relu")(x2)
x2 = layers.Dropout(0.3)(x2)

# Combine
x = layers.concatenate([x1, x2])
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.4)(x)
out = layers.Dense(len(classes), activation="softmax")(x)

# Build model
model = Model([img_in, meta_in], out, name="image_meta_model")



# Compile with gradient clipping and learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4, clipnorm=1.0),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "image_meta_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ meta (InputLayer)   │ (None, 18)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 18)        │         72 │ meta[0][0]        │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ image (InputLayer)  │ (None, 224, 224,  │          0 │ -                 │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 64)        │      1,216 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ efficientnetb0      │ (None, 1280)      │  4,049,571 │ image[0][0]       │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 64)        │          0 │ dense_12[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 1344)      │          0 │ efficientnetb0[0… │
│ (Concatenate)       │                   │            │ dropout_8[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_13 (Dense)    │ (None, 128)       │    172,160 │ concatenate_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 128)       │          0 │ dense_13[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 6)         │        774 │ dropout_9[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,223,793 (16.11 MB)

 Trainable params: 174,186 (680.41 KB)

 Non-trainable params: 4,049,607 (15.45 MB)

In [26]:
print("✅ Any NaNs in final train data?", train_df[feat_cols].isnull().any().any())


✅ Any NaNs in final train data? False


In [27]:
# Cell 5 ── Train (Improved Version)
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        "image_meta_best.keras",   # Save only the best
        save_best_only=True,
        monitor="val_accuracy",
        verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=2,
        min_lr=1e-6,
        verbose=1
    )
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=33,
    callbacks=callbacks
)




Epoch 1/33
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step - accuracy: 0.2888 - loss: 1.7759
Epoch 1: val_accuracy improved from -inf to 0.55892, saving model to image_meta_best.keras
74/74 ━━━━━━━━━━━━━━━━━━━━ 29s 302ms/step - accuracy: 0.2909 - loss: 1.7726 - val_accuracy: 0.5589 - val_loss: 1.2571 - learning_rate: 1.0000e-04
Epoch 2/33
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step - accuracy: 0.5467 - loss: 1.2526
Epoch 2: val_accuracy improved from 0.55892 to 0.56566, saving model to image_meta_best.keras
74/74 ━━━━━━━━━━━━━━━━━━━━ 21s 286ms/step - accuracy: 0.5468 - loss: 1.2528 - val_accuracy: 0.5657 - val_loss: 1.2153 - learning_rate: 1.0000e-04
Epoch 3/33
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step - accuracy: 0.5739 - loss: 1.2095
Epoch 3: val_accuracy improved from 0.56566 to 0.57239, saving model to image_meta_best.keras
74/74 ━━━━━━━━━━━━━━━━━━━━ 21s 276ms/step - accuracy: 0.5739 - loss: 1.2093 - val_accuracy: 0.5724 - val_loss: 1.1854 - learning_rate: 1.0000e-04
Epoch 4/33
74/74 ━━━━━━━━

In [28]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os


In [4]:
data_dir = "/mnt/ssd1/saumia/data/images/IMG_CLASSES"
class_names = sorted(os.listdir(data_dir))
num_classes = len(class_names)



In [5]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
)

train_gen_2 = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="training",
    shuffle=True,
    seed=42
)

val_gen_2 = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="validation",
    shuffle=False,
    seed=42
)

Found 21726 images belonging to 10 classes.
Found 5427 images belonging to 10 classes.


In [6]:
# --- 3. Compute class weights (handle imbalance) ---
labels = train_gen_2.classes
class_weights_2 = compute_class_weight(class_weight="balanced", classes=np.unique(labels), y=labels)
class_weights_dict_2 = {i: weight for i, weight in enumerate(class_weights_2)}


In [7]:
# --- 4. Model Definition (Model 2) ---
model_2 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])
model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



/home/saumia/miniconda3/envs/work1/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-05-22 23:24:39.537695: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [ ]:
import tensorflow as tf

# --- 5. Training ---
# Ensure the model is running on the GPU
with tf.device('/GPU:0'):
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    history_2 = model_2.fit(
        train_gen_2,
        validation_data=val_gen_2,
        epochs=35,
        class_weight=class_weights_dict_2,
        callbacks=[early_stop]
    )


Epoch 1/35
679/679 ━━━━━━━━━━━━━━━━━━━━ 0s 792ms/step - accuracy: 0.4302 - loss: 1.7993

/home/saumia/miniconda3/envs/work1/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


679/679 ━━━━━━━━━━━━━━━━━━━━ 591s 870ms/step - accuracy: 0.4302 - loss: 1.7992 - val_accuracy: 0.2349 - val_loss: 2.0937
Epoch 2/35
679/679 ━━━━━━━━━━━━━━━━━━━━ 591s 870ms/step - accuracy: 0.4891 - loss: 1.5734 - val_accuracy: 0.2782 - val_loss: 2.0078
Epoch 3/35
679/679 ━━━━━━━━━━━━━━━━━━━━ 590s 868ms/step - accuracy: 0.5194 - loss: 1.4854 - val_accuracy: 0.2729 - val_loss: 2.0458
Epoch 4/35
679/679 ━━━━━━━━━━━━━━━━━━━━ 595s 876ms/step - accuracy: 0.5307 - loss: 1.4303 - val_accuracy: 0.2851 - val_loss: 2.0911
Epoch 5/35
679/679 ━━━━━━━━━━━━━━━━━━━━ 593s 872ms/step - accuracy: 0.5398 - loss: 1.4055 - val_accuracy: 0.3077 - val_loss: 1.9759
Epoch 6/35
679/679 ━━━━━━━━━━━━━━━━━━━━ 594s 874ms/step - accuracy: 0.5589 - loss: 1.3456 - val_accuracy: 0.3121 - val_loss: 1.9722
Epoch 7/35
679/679 ━━━━━━━━━━━━━━━━━━━━ 592s 871ms/step - accuracy: 0.5780 - loss: 1.3073 - val_accuracy: 0.3118 - val_loss: 2.0330
Epoch 8/35
679/679 ━━━━━━━━━━━━━━━━━━━━ 589s 867ms/step - accuracy: 0.5938 - loss: 1.26

In [ ]:
# Save image model
model.save("./image_model.keras")
model_2.save("./image_model_2.keras")